In [2]:
%load_ext autoreload
%autoreload 2
%pdb
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)

Automatic pdb calling has been turned ON


In [1]:
import sys
#sys.path.insert(0, '/home/ubuntu/Documents/infembed/infembed')
sys.path.insert(0, '/home/ubuntu/Documents/infembed/infembed')
sys.path.insert(0, '/home/ubuntu/Documents/infembed/data')
from data._core.spotcheck import get_spotcheck_dataloader, get_blindspots_df
sys.path.insert(0, '/home/ubuntu/Documents/infembed/models')
#sys.path.insert(0, '/home/ubuntu/Documents/infembed/')
# sys.path.insert(0, )
from infembed.embedder._core.fast_kfac_embedder import FastKFACEmbedder
import torchvision
from torch.utils.data import Subset, DataLoader, default_collate, Dataset
from torchvision.models import ResNet18_Weights, resnet18
import torch.nn as nn
from infembed.clusterer._core.sklearn_clusterer import SklearnClusterer
from infembed.clusterer._core.rule_clusterer import RuleClusterer
from sklearn.cluster import KMeans
from tqdm import tqdm
import pandas as pd
import torch
from typing import List
from infembed.utils.common import Data
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Callable
from torch import Tensor
from typing import Tuple
from models._utils.common import init_linear
from lightning.pytorch.callbacks import EarlyStopping
import lightning.pytorch as pl
import lightning as L
import torch.nn.functional as F
import numpy as np

### figure out device to compute embeddings on ###

In [4]:
DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print('device:', DEVICE)

device: cuda:0


### define data
We will define the following:
- `eval_dataset`: `Dataset` for evaluation data.  This is used to retrieve individual examples for displaying.  This should be the same data for which the embeddings we load later were computed
- `eval_dataloader`: `DataLoader` for evaluation data.  This is used to compute metadata for the evaluation data

In [5]:
normalize = ResNet18_Weights.IMAGENET1K_V1.transforms()

def collate_fn(examples):
    return tuple([_x.to(device=DEVICE) for _x in default_collate([(normalize(__x[0]), __x[1]) for __x in examples])])

BATCH_SIZE = 32
NUM_EVAL = 50000
eval_dataset = Subset(
    torchvision.datasets.ImageNet("/home/ubuntu/Documents/infembed/data/files/imagenet", split="val"),
    range(NUM_EVAL),
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE)

### define model ###
only used to get metadata

In [6]:
model = resnet18(weights=ResNet18_Weights.DEFAULT).to(device=DEVICE)
# model.load_state_dict(ResNet18_Weights.IMAGENET1K_V1.get_state_dict(progress=True))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### load embeddings for evaluation data ###

In [64]:
# embeddings_path = '/home/ubuntu/Documents/infembed/infembed/scripts/run_embedder/outputs/imagenet_timing_experiments/0/embeddings.pt'
# embeddings_path = '/home/ubuntu/Documents/infembed/infembed/scripts/run_embedder/outputs/imagenet_kfac_global_projection_dim/1/embeddings.pt'
# embeddings_path = '/home/ubuntu/Documents/infembed/infembed/scripts/run_embedder/outputs/imagenet_kfac_last_layer/2/embeddings.pt'
# embeddings_path = '/home/ubuntu/Documents/infembed/infembed/scripts/run_embedder/outputs/imagenet_gradient_last_layer/0/embeddings.pt'
# embeddings_path = '/home/ubuntu/Documents/infembed/infembed/scripts/run_embedder/outputs/imagenet_dim_1000/0/embeddings.pt'
# embeddings_path = '/home/ubuntu/Documents/infembed/infembed/examples/imagenet/hydra_outputs/run_embeddings/imagenet_pca_gradient/4/embeddings.pt' # pca gradient 1000 dim
embeddings_path = '/home/ubuntu/Documents/infembed/infembed/examples/imagenet/hydra_outputs/run_embeddings/imagenet_arnoldi/3/embeddings.pt' # arnoldi 
embeddings = torch.load(open(embeddings_path, 'rb'))
embeddings.shape

torch.Size([50000, 195])

optionally, we can only keep some elements of the embedding (relevant for `ArnoldiEmbedder`)

In [65]:
DIMS_TO_KEEP = 50
if DIMS_TO_KEEP is not None:
    embeddings = embeddings[:, -DIMS_TO_KEEP:]

optionally, we can normalize the embeddings to have the same norm

In [66]:
NORMALIZE = False
if NORMALIZE:
    embeddings = embeddings / torch.linalg.norm(embeddings, dim=1)[:, None]

In [67]:
if True:
    # already assume embeddings fit in memory, so no point in using incremental methods
    pass

we then package them into a `Data` instance, which contains all kinds of data that could possibly be used to do the subsequent clustering, i.e. including tabular metadata as well.

In [68]:
data = Data(embeddings=embeddings)

### compute metadata for evaluation data ###
this will be the ingredient needed to display the clusters.  later on, it will also be used by the rule-based clusterer.  therefore, we also add it to the running `Data` instance for easy access.

In [69]:
def _get_predictions_and_labels(_model, dataloader):
    dfs = []
    for batch in tqdm(dataloader):
        prediction_prob = (
            torch.nn.functional.softmax(_model(*batch[:-1]), dim=1)
            .detach()
            .to(device="cpu")
        )
        prediction_label = torch.argmax(prediction_prob, dim=1).to(device="cpu")
        label = batch[-1].to(
            device="cpu"
        )  # assuming batch is a tensor.  if not, can check
        dfs.append(
            pd.DataFrame(
                {
                    "prediction_label": prediction_label,
                    "label": label,
                    "prediction_prob": list(prediction_prob.numpy()),
                }
            )
        )
    df = pd.concat(dfs, axis=0)
    df.index = list(range(len(df)))
    return df

if False:
    metadata = _get_predictions_and_labels(model, eval_dataloader)

In [70]:
data.metadata = metadata

### define clusterer ###

In [71]:
clusterer = SklearnClusterer(sklearn_clusterer=KMeans(n_clusters=25))

### do the clustering ###

In [72]:
clusters = clusterer.fit_predict(data)

### define ways to display clusters ###
these will all be functions whose input is a list of list of indices in the evaluation dataset

In [73]:
from infembed.visualization._core.common import (
    DisplayMetadata,
    DisplayPIL,
    DisplayPredictionAndLabels,
    DisplaySingleExamples,
    PerClusterDisplayer,
    DisplayAccuracy,
)

displayers = [
    PerClusterDisplayer(
        [
            DisplayAccuracy(prediction_col="prediction_label", label_col="label"),
            DisplayPredictionAndLabels(
                prediction_col="prediction_label", label_col="label"
            ),
            DisplaySingleExamples([
                DisplayMetadata(['label', 'prediction_label']),
                DisplayPIL(),
            ], limit=3),
        ]
    )
]

### display the clusters ###

In [74]:
for displayer in displayers:
    displayer(clusters, data)

cluster #0
accuracy: 0.70 (34046/48800)
prediction: {819: 92, 424: 90, 679: 82, 926: 81, 238: 79, 162: 77, 159: 76, 61: 75, 391: 75, 234: 74, 518: 72, 523: 72, 474: 71, 788: 71, 766: 70, 681: 70, 407: 69, 379: 69, 894: 69, 249: 69, 505: 68, 352: 68, 281: 68, 570: 68, 895: 67, 652: 67, 489: 67, 349: 67, 431: 67, 965: 66, 189: 66, 929: 66, 903: 66, 781: 65, 654: 65, 799: 65, 617: 65, 713: 65, 497: 64, 382: 64, 544: 64, 225: 64, 48: 64, 849: 64, 526: 64, 35: 64, 286: 64, 562: 63, 528: 63, 608: 63, 180: 63, 463: 63, 581: 63, 659: 63, 553: 63, 671: 63, 410: 63, 496: 63, 611: 62, 941: 62, 411: 62, 23: 62, 425: 62, 716: 62, 217: 62, 74: 62, 263: 62, 579: 62, 916: 62, 801: 62, 455: 62, 684: 61, 614: 61, 58: 61, 191: 61, 777: 61, 909: 61, 198: 61, 248: 61, 421: 61, 472: 61, 593: 61, 695: 61, 973: 61, 209: 60, 223: 60, 275: 60, 982: 60, 605: 60, 487: 60, 445: 60, 774: 60, 269: 60, 850: 60, 917: 60, 182: 60, 72: 60, 933: 60, 50: 60, 512: 60, 990: 60, 441: 60, 448: 60, 255: 60, 927: 60, 732: 59, 9

### define rule clusterer ###

In [75]:
def _accuracy(data):
    return (data.metadata["prediction_label"] == data.metadata["label"]).mean()


def _size(data):
    return len(data)


rule_clusterer = RuleClusterer(
    clusterer_getter=lambda n_clusters: SklearnClusterer(KMeans(n_clusters=n_clusters)),
    cluster_rule=lambda data: _accuracy(data) < 0.5 and _size(data) >= 10,
    stopping_rule=lambda data: _size(data) < 50,
    max_depth=7,
    branching_factor=3,
)

### do the rule clustering ###

In [76]:
rule_clusters = rule_clusterer.fit_predict(data)

### display the rule clusters

In [77]:
for displayer in displayers:
    displayer(rule_clusters, data)

cluster #0
accuracy: 0.40 (4/10)
prediction: {185: 2, 19: 1, 201: 1, 151: 1, 230: 1, 364: 1, 780: 1, 692: 1, 321: 1}
label: {12: 1, 154: 1, 158: 1, 185: 1, 191: 1, 230: 1, 364: 1, 780: 1, 921: 1, 946: 1}
